In [1]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Stats import WeightQuantization,IntroduceFaultsInWeights,GenerateFaultsList
from Nets import GetNeuralNetworkModel
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from datetime import datetime
import collections 
import pandas as pd



# Generación de una muestra de fallos

In [3]:
# Decidir si cargar errores de un archivo locs y error_mask o generarlos aleatoriamente
Cargar_errores = True
word_size  = 16

#print(load_file('Data/Fault Characterization/wgt/Accs_w_707_55'))

if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/locs_055')
    error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/error_mask_055')
    
else:
    numero_bits_con_fallo = 15000
    bits_con_fallo = np.random.randint(1,2,numero_bits_con_fallo)
    #crear una mascara (m) del buffer de pesos donde x: bit sin fallo, 0: bit con fallo en 0, 1: bit con fallo en 1.
    #si quieres introducir fallos en activaciones en lugar de los pesos, simplemente cambia wbuffer_size por abuffer_size.
    mbuffer = np.array(['x']*(abuffer_size-numero_bits_con_fallo))
    mbuffer = np.concatenate([mbuffer,bits_con_fallo])
    #distribuir los errores aleatoriamente en la mascara del buffer
    np.random.shuffle(mbuffer)
    #organizar la mascara del buffer por direcciones
    mbuffer = np.reshape(mbuffer,(-1,word_size))
    mbuffer = ["".join(i) for i in mbuffer]
    #filtrar dejando solo las direcciones con error
    locs  = [x for x,y in enumerate(mbuffer) if y.count('x') < 16]
    error_mask = [y for x,y in enumerate(mbuffer) if y.count('x') < 16] 
print('mostrando las 5 primeras direcciones con fallos')
print('direcciones:',locs[0:25])
print('mascara de fallos:',error_mask[0:25])
print(len(error_mask))
print(len(locs))


mostrando las 5 primeras direcciones con fallos
direcciones: [4141, 4181, 4241, 4789, 4899, 4903, 5001, 18073, 18199, 18343, 18355, 18365, 28702, 28894, 29150, 30576, 34604, 34666, 43076, 43658, 43810, 64280, 64428, 68938, 71185]
mascara de fallos: ['x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x1xxxxxxx0xxxxxx', 'x0xxxxxxx1xxxxxx', 'x1xxxxxxx1xxxxxx', 'x1xxxxxxx1xxxxxx', 'x1xxxxxxx1xxxxxx', 'x0xxxxxxx0xxxxxx', 'x1xxxxxxx1xxxxxx', 'x0xxxxxxx1xxxxxx', 'xx1xxxxxxx0xxxxx', 'xx1xxxxxxx1xxxxx', 'xx1xxxxxxx0xxxxx', 'x1xxxxxxx1xxxxxx', 'xx1xxxxxxx1xxxxx', 'xx1xxxxxxx1xxxxx', 'xxxx0xxxxxxx1xxx', 'xxxxx0xxxxxxx0xx', 'xxx0xxxxxxx0xxxx', 'x0xxxxxxx1xxxxxx', 'x1xxxxxxx0xxxxxx', 'xxxxxxx0xxxxxxx0', 'xxx1xxxxxxx0xxxx']
1599
1599


In [4]:
mask='1xxxxxxxx1xxx'
static_1_error  = int("".join(mask.replace('x','0')),)
static_0_Error  = int("".join(mask.replace('x','1')),)

In [5]:
static_1_error

1000000001000

In [6]:
static_0_Error

1111111111111

In [1]:
16777216/16

1048576.0

# Covetir la máscara de fallos en vectores de 0.

In [7]:
# Decidir si cargar errores de un archivo locs y error_mask o generarlos aleatoriamente
from Simulation import buffer_simulation, save_obj, load_obj
Cargar_errores = False
buffer_adress= (1024*1024)*2
word_size  = 16

#print(load_file('Data/Fault Characterization/wgt/Accs_w_707_55'))
vol=54
if Cargar_errores:
    locs  = load_obj('MoRS/Modelo3_mas_fallos_col_8_experimentos/mask/locs_0')
    #error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/error_mask_0_60')
    #locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-705/locs_0_54')
    #print('entre')
    #error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-705/error_mask_705_0_54')
else:
    #locs  = load_obj('Data/Fault Characterization/error_mask_30/vc_707/locs_0'+str(vol))
    locs  = load_obj('MoRS/Modelo3_mas_fallos_col_8_experimentos/mask/locs_0')
    print(len(locs))
    numero_bits_con_fallo = len(locs)
    bits_con_fallo = np.random.randint(0,2,numero_bits_con_fallo)
    mbuffer = np.array(['x']*(buffer_adress-numero_bits_con_fallo))
    mbuffer = np.concatenate([mbuffer,bits_con_fallo])
    #Convertirlo en vectores de 16 elementos
    mbuffer = np.reshape(mbuffer,(-1,word_size))
    mbuffer = ["".join(i) for i in mbuffer]
    error_mask=mbuffer[0:numero_bits_con_fallo]
    #more_posiction_errors=len(error_mask)-len(locs)
#save_obj(locs,'Data/Fault Characterization/error_mask_0x_30/vc_707/locs_0'+str(vol))
#save_obj(error_mask,'Data/Fault Characterization/error_mask_0x_30/vc_707/error_mask_0'+str(vol))
print('mostrando las 5 primeras direcciones con fallos')
print('direcciones:',locs[0:5])
print('mascara de fallos:',error_mask[0:100])
print(len(error_mask))
#print(more_posiction_errors)
print(len(locs))
#save_obj(locs,'Data/Fault Characterization/error_mask_0x_30/locs_0'+str(vol))
save_obj(error_mask,'MoRS/Modelo3_mas_fallos_col_8_experimentos/mask/error_mask_all_x')


68318
mostrando las 5 primeras direcciones con fallos
direcciones: [0, 1, 7, 8, 17]
mascara de fallos: ['xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxx

## Máscara de fallo a 0 con el fichero alterado 

In [9]:
# Decidir si cargar errores de un archivo locs y error_mask o generarlos aleatoriamente
from Simulation import buffer_simulation, save_obj, load_obj
import numpy as np
Cargar_errores = False
buffer_adress= (1024*1024)*1
word_size  = 16
#n_bits_fails = np.ceil(Enumber*network_size).astype(int)

#print(load_file('Data/Fault Characterization/wgt/Accs_w_707_55'))
vol=54
if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/locs_0' +str(vol))
    #error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/error_mask_0_60')
    #locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-705/locs_0_54')
    #print('entre')
    #error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-705/error_mask_705_0_54')
else:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/locs_0'+str(vol))
    print(len(locs))
    numero_bits_con_fallo = len(locs)
    bits_con_fallo = np.random.randint(0,1,numero_bits_con_fallo)
    mbuffer = np.array(['x']*(buffer_adress-numero_bits_con_fallo))
    mbuffer = np.concatenate([mbuffer,bits_con_fallo])
    #Convertirlo en vectores de 16 elementos
    mbuffer = np.reshape(mbuffer,(-1,word_size))
    mbuffer = ["".join(i) for i in mbuffer]
    error_mask=mbuffer[0:numero_bits_con_fallo]
    #more_posiction_errors=len(error_mask)-len(locs)
save_obj(locs,'Data/Fault Characterization/variante_mask_vc_707/error_mask_todo_x/vc_707/locs_0'+str(vol))
save_obj(error_mask,'Data/Fault Characterization/variante_mask_vc_707/error_mask_todo_x/vc_707/error_mask_0'+str(vol))
print('mostrando las 5 primeras direcciones con fallos')
print('direcciones:',locs[0:5])
print('mascara de fallos:',error_mask[100:200])
print(len(error_mask))
#print(more_posiction_errors)
print(len(locs))



9439
mostrando las 5 primeras direcciones con fallos
direcciones: [4141, 4142, 4181, 4182, 4199]
mascara de fallos: ['xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xx

In [31]:
#error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x5/vc_707/error_mask_054')
error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_0_x/error_mask_0_x5/vc_707/error_mask_054')
print(error_mask[0:200])

['0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000',

In [16]:
print(len(error_mask))
print(len(error_mask_0x5))

47195
47195


In [2]:
1024*1024*1

1048576


## Cargar muestra ya guardada

In [4]:
import random
from Simulation import buffer_simulation, save_obj, load_obj
Cargar_errores = True
buffer_adress= (1024*1024)*1
#print(abuffer_size)

#print(load_file('Data/Fault Characterization/wgt/Accs_w_707_55'))
vol=54

 

if Cargar_errores:
    locs = load_obj('Data/Fault Characterization/variante_mask_vc_707/only_VBW_with_error/vc_707/locs_0'+ str(vol))
    #locs=load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada_x/mask_volteada/vc_707/locs_054')
    print(len(locs))
    print('mayor valor',max(locs))
    print('minimo valor',min(locs))
    error_mask_ = load_obj( 'Data/Fault Characterization/variante_mask_vc_707/only_VBW_with_error/vc_707/error_mask_0'+ str(vol))
    #print(error_mask)
    #error_mask_=load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada_x/mask_volteada/vc_707/error_mask_054')
    
    #print(error_mask_)
    #locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_pesos_vc-707/locs_0_54_buffer_pesos')
    #error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_pesos_vc-707/error_mask_0_54_buffer_pesos')
#print((error_mask))    
error_mask=(error_mask_)*2
#print(new_error_mask)
#rand_num_total=len(error_mask)-len(error_mask_)
#print(rand_num_total)
print('tamaño de error_mask', len(error_mask_))
print('tamaño de new_error_mask', len(error_mask))    
random.seed(15)
print('locs antes',len(locs))
while len(locs)!= len(error_mask):
#for i in new_error_mask:
    new_locs= random.randint(0,buffer_adress )
    if new_locs not in (locs):
        locs.append(new_locs)
        #print(locs)
        
        
print('locs despues',len(locs)) 
print('mostrando las 5 primeras direcciones con fallos')
print('direcciones:',locs[0:10])
print('mascara de fallos:',error_mask)
print('tamaño de error_mask', len(error_mask)) 

#print('error_mask',error_mask)
save_obj(locs,'Data/Fault Characterization/variante_mask_vc_707/only_VBW_with_error/x2/locs_0'+str(vol))
save_obj(error_mask,'Data/Fault Characterization/variante_mask_vc_707/only_VBW_with_error/x2/error_mask_0'+str(vol))
 
#dup = [x for i, x in enumerate(locs) if i != locs.index(x)]
#print('dup',dup) 
##i=0
  


9439
mayor valor 1045319
minimo valor 4141
tamaño de error_mask 9439
tamaño de new_error_mask 18878
locs antes 9439
locs despues 18878
mostrando las 5 primeras direcciones con fallos
direcciones: [4141, 4142, 4181, 4182, 4199, 4200, 4241, 4242, 4249, 4250]
mascara de fallos: ['xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xxxxxxxxxxxxxxxx', 'xx

In [6]:
vol=54
locs=load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada_x/prueba/locs_0'+str(vol))
error_mask=load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada_x/prueba/error_mask_0'+str(vol))
print(error_mask)
 

['xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxx0xxxxxxx', 'xxxxxxxxxxxxxxx0', 'xxxxxxxx0xxxxxxx', 'xxxxxxxxxxxxxxx1', 'xxxxxxxxxxxxx10x', 'xxxxxxxxx00xxxxx', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxxxx0xxxx', 'xxxxxxxxxxxx1xxx',

## Analizar Accurancy inyectando errores en la última Capa
### Lo haré con la máscara de error del buffer de pesos para el voltaje 0.54

In [4]:
Cargar_errores = True
vol=54
if Cargar_errores:
    error_mask=load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada_30/error_mask_0'+str(vol))
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada_30/locs_0'+ str(vol))
    
    

print('mostrando las 5 primeras direcciones con fallos')
print('direcciones:',locs[0:5])
print('mascara de fallos:',error_mask[0:5])
print(len(error_mask))
print(len(locs))    

mostrando las 5 primeras direcciones con fallos
direcciones: [4141, 4142, 4181, 4182, 4199]
mascara de fallos: ['xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx']
283170
283170


## AlexNet

In [27]:
from Nets_original import GetNeuralNetworkModel
# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 11  
aint_size  = 4
wfrac_size = 11
wint_size  = 4
# Tamaño del buffer de pesos == al tamaño de la capa con mayor numero de pesos (885120 pesos de 16 bits cada uno)
wbuffer_size = 885120*word_size
# Tamaño del buffer de activaciones == al tamaño de la capa con mayor numero de activaciones (290400 pesos de 16 bits cada uno)
abuffer_size = (1024*1024)*word_size
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

acc_redes=[]

trainBatchSize = testBatchSize = 10
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)



#for i,layer in enumerate(Net1.layers):
#    #if Net1.layers[i].__class__.__name__== 'ReLU':
#    ##print('Capa',(i+2) ,Net1.layers[i+2].__class__.__name__)
#     #if Net1.layers[i].__class__.__name__=='Lambda':
#    activation_aging = [True,False,False,False,False,False,False,False,False,False]

#activation_agin = np.array([False]*11)
#acc_list=[]
##print(activation_aging)

#for i, valor in enumerate(activation_agin):
#    activation_agin[i]=True 
#    activation_agin[i-1]=False
#    print ('ciclo', i)
#    activation_aging= activation_agin
#    
#    print (activation_agin)
#activation_aging = [False,True,False,False,False,False,False,False,False,False,False,False]


Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8,faulty_addresses=locs, masked_faults=error_mask,
                        aging_active=True, word_size=word_size, frac_size=afrac_size, 
                        batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
print('AlexNet')
loss_s,acc_s  = Net2.evaluate(test_dataset)
acc_redes.append(acc_s)


estoy en WeightQuantization
AlexNet
75/75 [==============================] - 24s 299ms/step - loss: 8.9060 - accuracy: 0.1173


In [ ]:
tensor[5,6,7]  [40960,24576,57344]

In [8]:
bin(5)

'0b101'

In [9]:
int('1010000000000000',2)

40960

In [5]:
acc_redes

[0.890666663646698]

### inyectando error en ultimas capas

In [11]:
acc_s

0.36266666650772095

## SqueezeNet

In [28]:
from Nets_original import GetNeuralNetworkModel
# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 9 
aint_size  = 6
wfrac_size = 15
wint_size  = 0

# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'SqueezeNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')



trainBatchSize = testBatchSize = 10
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(224,224), 8, trainBatchSize, testBatchSize)



#for i,layer in enumerate(Net1.layers):
#    #if Net1.layers[i].__class__.__name__== 'ReLU':
#    ##print('Capa',(i+2) ,Net1.layers[i+2].__class__.__name__)
#     #if Net1.layers[i].__class__.__name__=='Lambda':
#    activation_aging = [True,False,False,False,False,False,False,False,False,False]

#activation_agin = np.array([False]*11)
#acc_list=[]
##print(activation_aging)

#for i, valor in enumerate(activation_agin):
#    activation_agin[i]=True 
#    activation_agin[i-1]=False
#    print ('ciclo', i)
#    activation_aging= activation_agin
#    
#    print (activation_agin)
#activation_aging = [False,True,False,False,False,False,False,False,False,False,False,False]

Net2 = GetNeuralNetworkModel('SqueezeNet', (224,224,3), 8,faulty_addresses=locs, masked_faults=error_mask,
                        aging_active=True, word_size=word_size, frac_size=afrac_size, 
                        batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
print('SqueezeNet')
loss_s,acc_s  = Net2.evaluate(test_dataset)
acc_redes.append(acc_s)

estoy en WeightQuantization
SqueezeNet
75/75 [==============================] - 78s 1s/step - loss: 12.0038 - accuracy: 0.1440


### error_mask en 0 error en ultimas capas

In [7]:
acc_redes


[0.890666663646698, 0.9306666851043701]

### inyectando error en ultimas capas

In [13]:
acc_s

0.12266666442155838

## DenseNet

In [6]:
from Nets_original import GetNeuralNetworkModel
# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 12 
aint_size  = 3
wfrac_size = 13
wint_size  = 2


abuffer_size = 16777216
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'DenseNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')



trainBatchSize = testBatchSize = 10
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(224,224), 8, trainBatchSize, testBatchSize)


#for i,layer in enumerate(Net1.layers):
#    #if Net1.layers[i].__class__.__name__== 'ReLU':
#    ##print('Capa',(i+2) ,Net1.layers[i+2].__class__.__name__)
#     #if Net1.layers[i].__class__.__name__=='Lambda':
#    activation_aging = [True,False,False,False,False,False,False,False,False,False]

#activation_agin = np.array([False]*11)
#acc_list=[]
##print(activation_aging)

#for i, valor in enumerate(activation_agin):
#    activation_agin[i]=True 
#    activation_agin[i-1]=False
#    print ('ciclo', i)
#    activation_aging= activation_agin
#    
#    print (activation_agin)
#activation_aging = [False,True,False,False,False,False,False,False,False,False,False,False]

Net2 = GetNeuralNetworkModel('DenseNet', (224,224,3), 8,faulty_addresses=locs, masked_faults=error_mask,
                        aging_active=True, word_size=word_size, frac_size=afrac_size, 
                        batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
print('DenseNet')
loss_s,acc_s  = Net2.evaluate(test_dataset)
acc_redes.append(acc_s)

estoy en WeightQuantization
DenseNet
75/75 [==============================] - 231s 3s/step - loss: 5.0630 - accuracy: 0.2427


NameError: name 'acc_redes' is not defined

In [17]:
### inyectando error en ultimas capas

In [18]:
acc_s

0.515999972820282

## MobileNet

In [5]:
from Nets_original import GetNeuralNetworkModel
# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 11 
aint_size  = 4
wfrac_size = 14
wint_size  = 1


abuffer_size = 16777216
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'MobileNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')



trainBatchSize = testBatchSize = 10
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(224,224), 8, trainBatchSize, testBatchSize)



Net2 = GetNeuralNetworkModel('MobileNet', (224,224,3), 8,faulty_addresses=locs, masked_faults=error_mask,
                        aging_active=True, word_size=word_size, frac_size=afrac_size, 
                        batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
print('MobileNet')
loss_s,acc_s  = Net2.evaluate(test_dataset)
acc_redes.append(acc_s)

estoy en WeightQuantization
MobileNet
75/75 [==============================] - 82s 1s/step - loss: 2.6991 - accuracy: 0.1187


NameError: name 'acc_redes' is not defined

### inyectando error en ultimas capas

In [27]:
acc_s

0.12266666442155838

## VGG16

In [14]:
from Nets_original import GetNeuralNetworkModel
# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 12 
aint_size  = 3
wfrac_size = 15
wint_size  = 0


abuffer_size = 16777216
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'VGG16')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')



trainBatchSize = testBatchSize = 10
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(224,224), 8, trainBatchSize, testBatchSize)


#for i,layer in enumerate(Net1.layers):
#    #if Net1.layers[i].__class__.__name__== 'ReLU':
#    ##print('Capa',(i+2) ,Net1.layers[i+2].__class__.__name__)
#     #if Net1.layers[i].__class__.__name__=='Lambda':
#    activation_aging = [True,False,False,False,False,False,False,False,False,False]

#activation_agin = np.array([False]*11)
#acc_list=[]
##print(activation_aging)

#for i, valor in enumerate(activation_agin):
#    activation_agin[i]=True 
#    activation_agin[i-1]=False
#    print ('ciclo', i)
#    activation_aging= activation_agin
#    
#    print (activation_agin)
#activation_aging = [False,True,False,False,False,False,False,False,False,False,False,False]

Net2 = GetNeuralNetworkModel('VGG16', (224,224,3), 8,faulty_addresses=locs, masked_faults=error_mask,
                        aging_active=True, word_size=word_size, frac_size=afrac_size, 
                        batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
print('VGG16')
loss_s,acc_s  = Net2.evaluate(test_dataset)
acc_redes.append(acc_s)

estoy en WeightQuantization
VGG16
75/75 [==============================] - 198s 3s/step - loss: 8.7522 - accuracy: 0.1173


In [ ]:
### inyectando error en ultimas capas

In [25]:
acc_s

0.11733333021402359

## ZFNet

In [15]:
from Nets_original import GetNeuralNetworkModel
# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 11 
aint_size  = 4
wfrac_size = 15
wint_size  = 0


abuffer_size = 16777216
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'ZFNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')



trainBatchSize = testBatchSize = 10
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(224,224), 8, trainBatchSize, testBatchSize)


#for i,layer in enumerate(Net1.layers):
#    #if Net1.layers[i].__class__.__name__== 'ReLU':
#    ##print('Capa',(i+2) ,Net1.layers[i+2].__class__.__name__)
#     #if Net1.layers[i].__class__.__name__=='Lambda':
#    activation_aging = [True,False,False,False,False,False,False,False,False,False]

#activation_agin = np.array([False]*11)
#acc_list=[]
##print(activation_aging)

#for i, valor in enumerate(activation_agin):
#    activation_agin[i]=True 
#    activation_agin[i-1]=False
#    print ('ciclo', i)
#    activation_aging= activation_agin
#    
#    print (activation_agin)
#activation_aging = [False,True,False,False,False,False,False,False,False,False,False,False]

Net2 = GetNeuralNetworkModel('ZFNet', (224,224,3), 8,faulty_addresses=locs, masked_faults=error_mask,
                        aging_active=True, word_size=word_size, frac_size=afrac_size, 
                        batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
print('ZFNet')
loss_s,acc_s  = Net2.evaluate(test_dataset)
acc_redes.append(acc_s)

estoy en WeightQuantization
ZFNet
75/75 [==============================] - 20s 258ms/step - loss: 9.7482 - accuracy: 0.1320


In [29]:
### inyectando error en ultimas capas

In [30]:
acc_s

0.2680000066757202

In [12]:
redes=['AlexNet', 'SqueezeNet', 'DenseNet','MobileNet', 'VGG16', 'ZFNet']
df_redes=pd.DataFrame(redes)
df_acc_0_1=pd.DataFrame(['0','0','0','0','0','0'])
df_s_error=pd.DataFrame(['0','0','0','0','0','0'])
df_acc_mask_0=pd.DataFrame(acc_redes)
df_redes=pd.DataFrame(redes)
df_mask_0=pd.concat([df_redes,df_acc_0_1,df_acc_mask_0, df_s_error], axis=1, join='outer')
df_mask_0.columns = ['Red','acc_0_1','acc_m_o','s_error']
df_mask_0.to_excel('mask_0.xlsx' ,sheet_name='vol', index=False)

In [13]:
df_mask_0

,Red,acc_0_1,acc_m_o,s_error
0,AlexNet,0,0.890667,0
1,SqueezeNet,0,0.930667,0
2,DenseNet,0,0.916000,0
3,MobileNet,0,0.880000,0
4,VGG16,0,0.805333,0
5,ZFNet,0,0.833333,0
